In [1]:
from cnn_image_generation import *
from analysis_nn import *
from update_library import *
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time

lib = update_library()

Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with

In [2]:
import squeezedet as nn
net = nn.init()

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.


InternalError: Failed to create session.

In [2]:
num_pics = 2000
# Sampling bounds (x,y)
sample_dim = 6
max_num_cars = 3
# images_domain = [[52,72],[1,19]]
images_domain = [[52,72],[1,19]]
modification_domain = [[0,1],[0.35,1],[0.8,1.2],[0.7,1]]

#path_synth_set = './synth/uniform/'
#path_synth_set = './synth/uniform_misclass/'

err_table = error_table()

In [4]:
def scale_sample(sample, sampling_domain):
    '''Scale a [0,1] sample in a given domain'''
    for i in range(len(sample)):
        sample[i] = sample[i]*(sampling_domain[i][1] - sampling_domain[i][0]) + sampling_domain[i][0]
    return sample    



def random_sample(typ, domain, n_samples):
    
    sample = []
    for _ in range(n_samples):
        if typ == 'float':
            r = random.random()*(domain[1]-domain[0])+domain[0]
        elif typ == 'int':
            r = random.choice(range(domain[0],domain[1]))
        else:
            print('Error')
        sample.append(r)    
    return sample

def random_config(domains, n_cars):
#     print("domains is : ", domains)# [[52, 72], [1, 19], [0, 1], [0.35, 1], [0.8, 1.2], [0.7, 1]]
#     print("n_cars is : ", n_cars)
    # Scene and cars
    sample = [random_sample('int',domains[0],1)]
    sample += [random_sample('int',domains[1],n_cars)]
    # Modifications
    x_sample = []
    y_sample = []
    if n_cars > 0:
        step_x = float(domains[2][1])/n_cars
        step_y = float(domains[3][1])/n_cars

        base_x = 0
        base_y = 0
        for _ in range(n_cars):
            x_sample.append(random.uniform(base_x, base_x+step_x))
            y_sample.append(random.uniform(base_y, base_y+step_y))
            base_x += step_x    
            base_y += step_y
            
    shuffle(x_sample)    
    y_sample.sort(reverse=True)
    sample += [x_sample]
    sample += [y_sample]

    sample += [random_sample('float',domains[4],1)]
    sample += [random_sample('float',domains[5],1)]
    
    return sample    
    

def box_2_kitti_format(box):
    '''Transform box for KITTI label format'''
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    left = int(x - w/2)
    right = int(x + w/2)
    top = int(y - h/2)
    bot = int(y + h/2)
    return [left,top,right,bot]

In [13]:
for batch in range(10000,11000,1000):
    
    for ratio in range(20,25,5):
        
        num_pics = batch/100*ratio
        path_synth_set = './synth/' + str(1500) + '/'
        
        print('Generating ' + str(num_pics) + '/' + str(batch) + '/' + str(ratio))

        i = 0
        v = 0

#         while i < num_pics:
        while i < 5000:

            num_cars = random.randint(1, max_num_cars)
            save_img = False
            try_num = 0
            while (not save_img) and (try_num < 25):
                sample = random_config(images_domain + modification_domain, num_cars)

                img, ground_boxes, img_info = gen_comp_img(lib, bg_id=sample[0][0], fg_id=sample[1], x=sample[2], y=sample[3], brightness=sample[4][0], color=sample[5][0])

                # Save tmp image    

    #             tmp_img_file_name = './tmp.png'
    #             img.save(tmp_img_file_name)

    #             # Classify img
    #             (boxes,probs,cats) = nn.classify(tmp_img_file_name,net) 

                save_img = True
                for m in range(len(ground_boxes)):
                    for n in range(m+1, len(ground_boxes)):
                        if nn.iomin(ground_boxes[m], ground_boxes[n]) > 0.4:
    #                         print("BAD")
                            save_img = False
                try_num += 1
                        
                    
#             # Not cars
#             if any(cats):
#                 save_img = True
#             else:
#                 for ground_box in ground_boxes:
#                     for box, prob in zip(boxes, probs):
#                         if( nn.iou(ground_box, box)*prob) < 0.25:
#                             save_img = True
            
            if save_img:
                file_name = 'm' + str(i).zfill(6)

                # Save image
                img_file_name = path_synth_set + 'images/' + file_name + '.png'
                img.save(img_file_name)
#                 print("Saving to ", img_file_name)

                # Save label
                f = open(path_synth_set + 'labels/' + file_name + '.txt', 'w')

                if len(ground_boxes) > 0:
                    for box in ground_boxes[:-1]:
                        label = [0,0,0] + box_2_kitti_format(box) + [0,0,0,0,0,0,0]
                        label = list(map(str, label))
                        label = " ".join(label)
                        label  = "Car " + label + "\n"
                        f.write(label)  # python will convert \n to os.linesep
                    label = [0,0,0] + box_2_kitti_format(ground_boxes[-1]) + [0,0,0,0,0,0,0]
                    label = list(map(str, label))
                    label = " ".join(label)
                    label  = "Car " + label
                    f.write(label)  # python will convert \n to os.linesep
                f.close()

        # #         err_table.update_with_elem(img_info)

                i+=1
            v+=1

            if(v %50 == 0):
                print('i: ' + str(i) + '/' + 'h: ' + str(float(i)/float(v)))

Generating 2000/10000/20
i: 38/h: 0.76
i: 70/h: 0.7
i: 107/h: 0.713333333333
i: 143/h: 0.715
i: 182/h: 0.728
i: 221/h: 0.736666666667
i: 261/h: 0.745714285714
i: 298/h: 0.745
i: 336/h: 0.746666666667
i: 378/h: 0.756
i: 414/h: 0.752727272727
i: 450/h: 0.75
i: 489/h: 0.752307692308
i: 523/h: 0.747142857143
i: 564/h: 0.752
i: 599/h: 0.74875
i: 636/h: 0.748235294118
i: 674/h: 0.748888888889
i: 715/h: 0.752631578947
i: 756/h: 0.756
i: 797/h: 0.759047619048
i: 839/h: 0.762727272727
i: 879/h: 0.764347826087
i: 918/h: 0.765
i: 957/h: 0.7656
i: 996/h: 0.766153846154
i: 1031/h: 0.763703703704
i: 1069/h: 0.763571428571
i: 1113/h: 0.767586206897
i: 1150/h: 0.766666666667
i: 1193/h: 0.769677419355
i: 1224/h: 0.765
i: 1262/h: 0.764848484848
i: 1302/h: 0.765882352941
i: 1343/h: 0.767428571429
i: 1383/h: 0.768333333333
i: 1425/h: 0.77027027027
i: 1468/h: 0.772631578947
i: 1504/h: 0.771282051282
i: 1542/h: 0.771
i: 1579/h: 0.770243902439
i: 1621/h: 0.771904761905
i: 1662/h: 0.773023255814
i: 1700/h: 0.

In [102]:
def gen_image_set(n, k):
    im_set = random.sample(range(0,n),  k)
    with open('3000_005.txt',"w") as fw:
        for im in im_set:
            fw.write('m' + str(im).zfill(6) + "\n")    

In [21]:
lst = list(range(0,10000))

for m in range(3000,11000,1000):
        
    random.shuffle(lst)
    train_set = lst[0:m]
    
    file_name = "train_" + str(m) + '_' + '0.txt'
    
    with open(file_name,"w") as fw:
        for x in train_set:
            fw.write(str(x).zfill(6) + "\n")

hello
hello
hello
hello
hello
hello
hello
hello


In [24]:
gen_image_set(num_pics,600)

In [1]:
sample

NameError: name 'sample' is not defined

In [44]:
sort(random_sample('float',[1,10],2))

NameError: name 'sort' is not defined

In [37]:
a = [[1]]

In [42]:
a [random_sample('float',[1,10],2)]

In [21]:
range(0,10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [12]:
a = [1,2]
b=['a','b']
print(a+b)

[1, 2, 'a', 'b']


In [91]:
print(random_sample('float',[1,10],2).sort(reverse=True))

None


In [115]:
print(random_config(images_domain + modification_domain, 3))

[[56], [2, 6, 8], None, None, [0.9009242275608164], [0.966690530132202]]


In [30]:
x = []

In [106]:
list1 = [1,2,3,4,5]
shuffle(list1)
print(list1)

[1, 3, 4, 2, 5]


In [88]:
a=[1,2,3]
b=[1,2,4]
a==b

False

In [ ]:
step = float(domains[1][1])/n_cars
    base = 0
    for _ in range(n_cars):
        x_sample.append(random.uniform(base, base+step))
        base += step